In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json

In [2]:
# [1]:
url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjce/_search" # URL do Tribunal de Justiça do Estado
api_key = "APIKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw==" # Sua Chave pública
headers = {
  'Authorization': api_key,
  'Content-Type': 'application/json'
}

# Usaremos o código 3619 (Assunto: Crimes contra a Fauna/Meio Ambiente)
# Este é um dos códigos mais prováveis de retornar o que você busca, dada a inconsistência do TJRJ.
CODIGO_CRIMES_AMBIENTAIS = 3619 
LIMITE_BUSCA = 3000

In [3]:
# [2]:
import json

payload = json.dumps({
  "size": LIMITE_BUSCA,
  "query": {
    # Filtra por Assunto 3619 (Crimes contra a Fauna/Meio Ambiente)
    "match": {"assuntos.codigo": CODIGO_CRIMES_AMBIENTAIS} 
  },
  # Ordena pela data mais recente (garantindo os julgados mais novos)
  "sort": [{"dataAjuizamento": {"order": "desc"}}]
})

In [5]:
# [3]:
import requests

response = requests.request("POST", url, headers=headers, data=payload)

# Verifica se a requisição foi bem-sucedida (Status 200)
if response.status_code == 200:
    dados_dict = response.json()
    
    # Total de processos que o TJCE tem para este Assunto (no DataJud)
    total_encontrado = dados_dict.get('hits', {}).get('total', {}).get('value', 0)
    
    # Lista dos processos baixados (até 700)
    processos_baixados = dados_dict.get('hits', {}).get('hits', [])
    
    print(f"Status da Resposta: {response.status_code}")
    print(f"Total de processos ENCONTRADOS pela API (Assunto 3619): {total_encontrado}")
    print(f"Total de processos BAIXADOS nesta requisição (máx. 3000): {len(processos_baixados)}")
    
    # Analisamos o primeiro processo para validar o Assunto:
    if processos_baixados:
        primeiro_processo = processos_baixados[0]['_source']
        assuntos_encontrados = primeiro_processo.get('assuntos', [])
        
        print("\n--- ANÁLISE DO PRIMEIRO PROCESSO ---")
        print(f"Classe: {primeiro_processo.get('classe', {}).get('nome')}")
        print("Assuntos:")
        for assunto in assuntos_encontrados:
            print(f"  - Código: {assunto.get('codigo')} | Nome: {assunto.get('nome')}")

else:
    print(f"Erro na Requisição. Status: {response.status_code}. Mensagem: {response.text}")

Status da Resposta: 200
Total de processos ENCONTRADOS pela API (Assunto 3619): 3417
Total de processos BAIXADOS nesta requisição (máx. 3000): 3000

--- ANÁLISE DO PRIMEIRO PROCESSO ---
Classe: Inquérito Policial
Assuntos:
  - Código: 3619 | Nome: Crimes contra a Fauna


In [6]:

# Extrai o campo '_source' de cada processo e cria o DataFrame
dados_fauna = [p['_source'] for p in processos_baixados]
df_processos = pd.DataFrame(dados_fauna)

print(f"Número de processos no DataFrame: {len(df_processos)}")
# Exibe as colunas para ter uma visão geral da estrutura
print("\nColunas do DataFrame:")
print(df_processos.columns.tolist())

Número de processos no DataFrame: 3000

Colunas do DataFrame:
['classe', 'numeroProcesso', 'sistema', 'formato', 'tribunal', 'dataHoraUltimaAtualizacao', 'grau', '@timestamp', 'dataAjuizamento', 'movimentos', 'id', 'nivelSigilo', 'orgaoJulgador', 'assuntos']


In [12]:
# [13]: Análise Detalhada dos Assuntos (Crimes Específicos)

from collections import Counter
import itertools

# 1. Extrair a lista de nomes de assuntos de CADA processo
# O campo 'assuntos' é uma lista de dicionários.
lista_de_listas_assuntos = df_processos['assuntos'].apply(
    lambda lista: [assunto.get('nome') for assunto in lista if isinstance(assunto, dict)]
).tolist()

# 2. "Achatar" a lista de listas para ter uma lista única de todos os nomes de assuntos
todos_os_assuntos = list(itertools.chain.from_iterable(lista_de_listas_assuntos))

# 3. Contar a frequência de cada assunto
contagem_assuntos = Counter(todos_os_assuntos)

print("\n" + "="*50)
print("TOP 10 ASSUNTOS ESPECÍFICOS (Teor dos Processos)")
print("="*50)

# Exibe os 10 assuntos mais comuns (o teor do crime ou tema principal)
top_10_assuntos = contagem_assuntos.most_common(10)

# Formata e imprime o resultado
for assunto, contagem in top_10_assuntos:
    print(f"- {assunto}: {contagem} ocorrências")


TOP 10 ASSUNTOS ESPECÍFICOS (Teor dos Processos)
- Crimes contra a Fauna: 3000 ocorrências
- Crimes do Sistema Nacional de Armas: 283 ocorrências
- Tráfico de Drogas e Condutas Afins: 157 ocorrências
- Intimação: 43 ocorrências
- Receptação: 43 ocorrências
- Associação para a Produção e Tráfico e Condutas Afins: 28 ocorrências
- Diligências: 26 ocorrências
- Ameaça : 19 ocorrências
- Posse de Drogas para Consumo Pessoal: 17 ocorrências
- Maus Tratos: 17 ocorrências


In [13]:
# [16]: Visualização COMPLETA do DataFrame

# 1. Lista todas as colunas existentes no DataFrame df_processos
print("Lista COMPLETA de todas as colunas no DataFrame df_processos:")
print(df_processos.columns.tolist())

# 2. Exibe uma amostra aleatória de 3 processos, incluindo TODAS as colunas.
# É necessário configurar as opções de exibição do Pandas para garantir que
# ele não corte o conteúdo das colunas, especialmente listas/dicionários grandes.
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

print("\n" + "="*100)
print("AMOSTRA ALEATÓRIA DE 3 PROCESSOS (Todas as 18 Colunas Incluídas)")
print("="*100)

# Usamos apenas 3 amostras e to_string() para melhor visualização,
# pois o DataFrame completo é muito largo.
print(df_processos.sample(1).to_string())

# Resetar as opções do Pandas (boa prática)
pd.reset_option('display.max_columns')
pd.reset_option('display.max_colwidth')

Lista COMPLETA de todas as colunas no DataFrame df_processos:
['classe', 'numeroProcesso', 'sistema', 'formato', 'tribunal', 'dataHoraUltimaAtualizacao', 'grau', '@timestamp', 'dataAjuizamento', 'movimentos', 'id', 'nivelSigilo', 'orgaoJulgador', 'assuntos']

AMOSTRA ALEATÓRIA DE 3 PROCESSOS (Todas as 18 Colunas Incluídas)
                                                classe        numeroProcesso                       sistema                              formato tribunal dataHoraUltimaAtualizacao grau                      @timestamp dataAjuizamento                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [15]:
# [19]: Gravar o DataFrame em um arquivo CSV

# Nome que você deseja dar ao arquivo
nome_arquivo = 'processos_crimes_fauna_tjce.csv'

try:
    # 1. Utilizamos o método .to_csv()
    # index=False: Garante que a coluna de índice do Pandas (0, 1, 2, ...) não seja salva no arquivo.
    # encoding='utf-8': É a codificação mais comum e garante que caracteres especiais (acentos) sejam salvos corretamente.
    df_processos.to_csv(nome_arquivo, index=False, encoding='utf-8')
    
    print("="*80)
    print(f"SUCESSO! O arquivo '{nome_arquivo}' foi gravado.")
    print(f"O arquivo contém {len(df_processos)} processos e {len(df_processos.columns)} colunas.")
    print("Você pode encontrar este arquivo no mesmo diretório do seu Notebook.")
    print("="*80)
    
except Exception as e:
    print(f"ERRO ao gravar o arquivo CSV: {e}")

SUCESSO! O arquivo 'processos_crimes_fauna_tjce.csv' foi gravado.
O arquivo contém 3000 processos e 14 colunas.
Você pode encontrar este arquivo no mesmo diretório do seu Notebook.


In [11]:
# [20]: Executar o df.info()

print("="*80)
print("INFORMAÇÕES GERAIS DO DATAFRAME df_processos (df.info())")
print("="*80)

# O .info() não retorna a informação como uma string que o print possa exibir diretamente
# em um ambiente como o meu, mas no seu Notebook ele imprimirá a tabela diretamente.
df_processos.info()

INFORMAÇÕES GERAIS DO DATAFRAME df_processos (df.info())
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   classe                     3000 non-null   object
 1   numeroProcesso             3000 non-null   object
 2   sistema                    3000 non-null   object
 3   formato                    3000 non-null   object
 4   tribunal                   3000 non-null   object
 5   dataHoraUltimaAtualizacao  3000 non-null   object
 6   grau                       3000 non-null   object
 7   @timestamp                 3000 non-null   object
 8   dataAjuizamento            3000 non-null   object
 9   movimentos                 3000 non-null   object
 10  id                         3000 non-null   object
 11  nivelSigilo                3000 non-null   int64 
 12  orgaoJulgador              3000 non-null   object
 13  assunt